# What's the highest selling game in the world?

With the dataset of games on Steam, we're curious about what is highest selling game in the world and what pattern, if any, is predicative of its success? 

## Data Pre-processing 

In [35]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib

spiderman_data = pd.read_csv('games.csv')

col_names = ['AppID','Name','Release date','Estimated owners','Required age','Price','Developers','Publishers','Genres']
#preprocessing for data duplicates
df = pd.DataFrame(spiderman_data)
df = df.drop_duplicates()
df = df[col_names]
df.head(10)
# for col in spiderman_data.columns:
#     print(col)

,AppID,Name,Release date,Estimated owners,Required age,Price,Developers,Publishers,Genres
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,19.99,Perpetual FX Creative,Perpetual FX Creative,"Casual,Indie,Sports"
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0.99,Rusty Moyher,Wild Rooster,"Action,Indie"
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,4.99,Campião Games,Campião Games,"Action,Adventure,Indie,Strategy"
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,5.99,Odd Critter Games,Odd Critter Games,"Adventure,Casual,Indie"
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0.00,Unusual Games,Unusual Games,"Adventure,Indie"
5,1469160,Wartune Reborn,"Feb 26, 2021",50000 - 100000,0,0.00,7Road,7Road,"Adventure,Casual,Free to Play,Massively Multip..."
6,1659180,TD Worlds,"Jan 9, 2022",0 - 20000,0,10.99,MAKSIM VOLKAU,MAKSIM VOLKAU,"Indie,Strategy"
7,1968760,Legend of Rome - The Wrath of Mars,"May 5, 2022",0 - 20000,0,9.99,magnussoft,magnussoft,Casual
8,1178150,MazM: Jekyll and Hyde,"Apr 2, 2020",0 - 20000,0,14.99,Growing Seeds,"CFK Co., Ltd.","Adventure,RPG,Simulation,Strategy"
9,320150,Deadlings: Rotten Edition,"Nov 11, 2014",50000 - 100000,0,3.99,ONE MORE LEVEL,ONE MORE LEVEL,"Action,Adventure,Indie"


We're interested in the sales number which is how much revenue the game brought in. However, we're not given that data but what we're given is the price for a copy of that game and estimated amount of owners. Therefore, we can compute the range of revenue for that game since we're given a lower bound and upper bound for estimated number of owners. 

In [145]:
sale_data = df[['Name','Estimated owners','Price','Genres']]

#extracted the EST owners into 2 columns
extracted_col = sale_data['Estimated owners'].str.extract('(\d+)\s-\s(\d+)')
extracted_col.columns = extracted_col.columns.map(str)
extracted_col.columns.values[0] = "Low Owners"
extracted_col.columns.values[1] = "High Owners"
#convert values to int for computation
extracted_col['Low Owners'] = pd.to_numeric(extracted_col['Low Owners'])
extracted_col['High Owners'] = pd.to_numeric(extracted_col['High Owners'])
#concatenate the extracted columsn back into the main dataframe
sale_data = pd.concat([sale_data,extracted_col],axis=1)
sale_data = sale_data.drop('Estimated owners',axis=1)
sale_data.info()
sale_data.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83560 entries, 0 to 83559
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         83554 non-null  object 
 1   Price        83560 non-null  float64
 2   Genres       80135 non-null  object 
 3   Low Owners   83560 non-null  int64  
 4   High Owners  83560 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 3.2+ MB


,Name,Price,Genres,Low Owners,High Owners
0,Galactic Bowling,19.99,"Casual,Indie,Sports",0,20000
1,Train Bandit,0.99,"Action,Indie",0,20000
2,Jolt Project,4.99,"Action,Adventure,Indie,Strategy",0,20000
3,Henosis™,5.99,"Adventure,Casual,Indie",0,20000
4,Two Weeks in Painland,0.00,"Adventure,Indie",0,20000
5,Wartune Reborn,0.00,"Adventure,Casual,Free to Play,Massively Multip...",50000,100000
6,TD Worlds,10.99,"Indie,Strategy",0,20000
7,Legend of Rome - The Wrath of Mars,9.99,Casual,0,20000
8,MazM: Jekyll and Hyde,14.99,"Adventure,RPG,Simulation,Strategy",0,20000
9,Deadlings: Rotten Edition,3.99,"Action,Adventure,Indie",50000,100000


Since the original column of "Estimated owners" are object type, I have to separate the range into lower and upper bound in order to compute the range for revenue. Furthermore, I had to convert the two columns' values to integer type for computation. 

## Data Generation & Analysis

Based on the previous step, I'm going to compute the range for revenue for each game and rank them base on the revenue spread with descending order for lower and upper bound of revenue. 

However, since games do run on two business models of paid games versus freemium games, we also take a look at the volume of copies of games that may be free and make revenue through in-game microtransactions that isn’t reflected in this dataset. 

### Revenue Chart

In [148]:
sale_data['Revenue Low'] = sale_data['Price'] * sale_data['Low Owners']
sale_data['Revenue High'] = sale_data['Price'] * sale_data['High Owners']
sale_data = sale_data.sort_values(by=['Revenue High','Revenue Low'],ascending=[False,False])
sale_data.head(20)

,Name,Price,Genres,Low Owners,High Owners,Revenue Low,Revenue High
7030,New World,39.99,"Action,Adventure,Massively Multiplayer,RPG",50000000,100000000,1.999500e+09,3.999000e+09
32756,ELDEN RING,59.99,"Action,RPG",20000000,50000000,1.199800e+09,2.999500e+09
35001,Rust,39.99,"Action,Adventure,Indie,Massively Multiplayer,RPG",20000000,50000000,7.998000e+08,1.999500e+09
8009,Cyberpunk 2077,59.99,RPG,10000000,20000000,5.999000e+08,1.199800e+09
18378,Mount & Blade II: Bannerlord,49.99,"Action,RPG,Simulation,Strategy,Early Access",10000000,20000000,4.999000e+08,9.998000e+08
8256,Tom Clancy's Rainbow Six® Siege,19.99,Action,20000000,50000000,3.998000e+08,9.995000e+08
33384,Valheim,19.99,"Action,Adventure,Indie,RPG,Early Access",20000000,50000000,3.998000e+08,9.995000e+08
53715,The Witcher® 3: Wild Hunt,39.99,RPG,10000000,20000000,3.999000e+08,7.998000e+08
79530,Starfield,69.99,RPG,5000000,10000000,3.499500e+08,6.999000e+08
1637,Sekiro™: Shadows Die Twice - GOTY Edition,59.99,"Action,Adventure",5000000,10000000,2.999500e+08,5.999000e+08


As we see in the chart above, the best selling game in term of revenue is New World with lower revenue bound of approximately 2 billion dollars and upper revenue bound of 4 billion dollars. Furthermore, within the top 20 games, where the lowest revenue bound is 2 billion, there's a correlation that if the game genre is either Action or RPG, it is more likely that the revenue for that game is greater and thus, the game will rank higher in this revenue chart. This may mean that more people enjoy those two genres of game the most on Steam versus games of other genres. 

### Volume Chart

In [149]:
sale_data = sale_data.sort_values(by=['Low Owners','High Owners'],ascending=[False,False])
sale_data.head(20)

,Name,Price,Genres,Low Owners,High Owners,Revenue Low,Revenue High
17585,Dota 2,0.00,"Action,Free to Play,Strategy",100000000,200000000,0.000000e+00,0.000000e+00
7030,New World,39.99,"Action,Adventure,Massively Multiplayer,RPG",50000000,100000000,1.999500e+09,3.999000e+09
8885,PUBG: BATTLEGROUNDS,0.00,"Action,Adventure,Free to Play,Massively Multip...",50000000,100000000,0.000000e+00,0.000000e+00
30583,Team Fortress 2,0.00,"Action,Free to Play",50000000,100000000,0.000000e+00,0.000000e+00
46158,Counter-Strike: Global Offensive,0.00,"Action,Free to Play",50000000,100000000,0.000000e+00,0.000000e+00
32756,ELDEN RING,59.99,"Action,RPG",20000000,50000000,1.199800e+09,2.999500e+09
35001,Rust,39.99,"Action,Adventure,Indie,Massively Multiplayer,RPG",20000000,50000000,7.998000e+08,1.999500e+09
8256,Tom Clancy's Rainbow Six® Siege,19.99,Action,20000000,50000000,3.998000e+08,9.995000e+08
33384,Valheim,19.99,"Action,Adventure,Indie,RPG,Early Access",20000000,50000000,3.998000e+08,9.995000e+08
1289,Garry's Mod,9.99,"Indie,Simulation",20000000,50000000,1.998000e+08,4.995000e+08


From looking at this volume chart containing the top 20 best "selling" games by volume, we can see a lot of changes from the previous chart of revenue-based. There are 9/20 games in the top 20 that are free to play and pull in a similar, if not greater amount of owners and players. Thus, the best-selling game based on volume is Dota 2 which boasts 100 to 200 million owners. We can observe another trend that within the top 20, free to play games have 9 games and paid games have 11 games, which is about the same. This may mean that as long as the game is good, people are willing to pay money for it. Vice versa, free to play games have an edge in volume due to the fact that it doesn't "cost" owners anything to get started.